<a href="https://colab.research.google.com/github/jongoh-Jeong/MachineLearning-for-AI/blob/main/1006.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import pandas as pd
import numpy as np

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


# Task 1

In [3]:
cd /content/drive/MyDrive/Hands-On-Data-Analysis-with-Pandas-2nd-edition/ch_04

/content/drive/MyDrive/Hands-On-Data-Analysis-with-Pandas-2nd-edition/ch_04


In [4]:
weather = pd.read_csv('data/weather_by_station.csv', index_col='date', parse_dates=True)
weather.head()

,datatype,station,value,station_name
date,,,,
2018-01-01,PRCP,GHCND:US1CTFR0039,0.0,"STAMFORD 4.2 S, CT US"
2018-01-01,PRCP,GHCND:US1NJBG0015,0.0,"NORTH ARLINGTON 0.7 WNW, NJ US"
2018-01-01,SNOW,GHCND:US1NJBG0015,0.0,"NORTH ARLINGTON 0.7 WNW, NJ US"
2018-01-01,PRCP,GHCND:US1NJBG0017,0.0,"GLEN ROCK 0.7 SSE, NJ US"
2018-01-01,SNOW,GHCND:US1NJBG0017,0.0,"GLEN ROCK 0.7 SSE, NJ US"


In [5]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 80256 entries, 2018-01-01 to 2018-12-31
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   datatype      80256 non-null  object 
 1   station       80256 non-null  object 
 2   value         80256 non-null  float64
 3   station_name  80256 non-null  object 
dtypes: float64(1), object(3)
memory usage: 3.1+ MB


### 1.Find which quarter had the most rain from the daily precipitation data.

In [6]:
dp = weather.query('datatype=="PRCP"').groupby(level=0).sum()

dp.head()

,value
date,
2018-01-01,0.0
2018-01-02,0.0
2018-01-03,0.0
2018-01-04,275.3
2018-01-05,689.0


In [7]:
data = dp.groupby(pd.Grouper(freq="Q")).sum().sort_values('value')

data.head()

,value
date,
2018-06-30,27086.3
2018-03-31,27669.5
2018-12-31,38571.3
2018-09-30,44436.5


In [8]:
dp.head()

,value
date,
2018-01-01,0.0
2018-01-02,0.0
2018-01-03,0.0
2018-01-04,275.3
2018-01-05,689.0


In [9]:
dp.assign(
      total_cnt_days=lambda x: x.groupby(pd.Grouper(freq = 'M')).value.transform('count'),
      quarter_total_precipitation= lambda x :x.groupby(pd.Grouper(freq = 'Q')).value.transform(np.sum)
)

,value,total_cnt_days,quarter_total_precipitation
date,,,
2018-01-01,0.0,31,27669.5
2018-01-02,0.0,31,27669.5
2018-01-03,0.0,31,27669.5
2018-01-04,275.3,31,27669.5
2018-01-05,689.0,31,27669.5
...,...,...,...
2018-12-27,0.0,31,38571.3
2018-12-28,826.5,31,38571.3
2018-12-29,1417.8,31,38571.3


In [10]:
dp.assign(
    rolling_PRCP = lambda x : x.value.rolling('3D').mean()
).T

date,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08,2018-01-09,2018-01-10,...,2018-12-22,2018-12-23,2018-12-24,2018-12-25,2018-12-26,2018-12-27,2018-12-28,2018-12-29,2018-12-30,2018-12-31
value,0.0,0.0,0.0,275.300000,689.000000,2.3,0.000000,7.200000,22.100000,0.000000,...,1437.1,6.8,168.500000,14.500000,0.6,0.000000,826.5,1417.8,4.700000,301.400000
rolling_PRCP,0.0,0.0,0.0,91.766667,321.433333,322.2,230.433333,3.166667,9.766667,9.766667,...,1513.4,1496.3,537.466667,63.266667,61.2,5.033333,275.7,748.1,749.666667,574.633333


# Task 2

In [11]:
fb = pd.read_csv('data/fb_2018.csv', index_col='date', parse_dates=True).assign(
    trading_volume=lambda x: pd.cut(x.volume, bins=3, labels=['low', 'med', 'high'])
)
fb.head()

,open,high,low,close,volume,trading_volume
date,,,,,,
2018-01-02,177.68,181.58,177.5500,181.42,18151903,low
2018-01-03,181.88,184.78,181.3300,184.67,16886563,low
2018-01-04,184.90,186.21,184.0996,184.33,13880896,low
2018-01-05,185.59,186.90,184.9300,186.85,13574535,low
2018-01-08,187.20,188.90,186.3300,188.28,17994726,low


In [12]:
fb.pivot_table(index='trading_volume')

,close,high,low,open,volume
trading_volume,,,,,
low,171.431771,173.461040,169.306252,171.355565,2.454721e+07
med,175.143750,179.420000,172.107500,175.818125,7.907256e+07
high,168.156667,170.476667,161.573333,167.726667,1.419240e+08


In [13]:
fb.pivot_table(
    index="trading_volume", columns=fb.index.month, values="close", aggfunc="count")

date,1,2,3,4,5,6,7,8,9,10,11,12
trading_volume,,,,,,,,,,,,
low,20,19,15,20,22,21,18,23,19,23,21,19
med,1,0,4,1,0,0,2,0,0,0,0,0
high,0,0,2,0,0,0,1,0,0,0,0,0
